# 1) Datasets

Vamos a leer el dataset que envía PyP

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('/mnt/c/users/Dleg/Data/MOPC/MOPC/Enriquecimiento de Datos/Data Cleaning/Contacto Directo/Datasets/Datasets Recibidos/Cartera_Completa Devuelta.xlsx',sheet_name='Emails')

In [3]:
df.head()

,Documento,Email
0,10128300,raulblanco11@hotmail.com
1,10141823,carmenotanez@hotmail.com
2,10460610,juando@gmail.com
3,10497875,caty1951@hotmail.com
4,10580772,fernando.bazan@hotmail.com


In [4]:
df.shape

(876, 2)

## 1.2) Data Cleaning: Datasets

Vamos a dropear aquellas columnas y filas que no tengan ningún valor (es decir, que sean todos NaNs)

In [5]:
df = df.dropna(axis=0, how='all')

In [6]:
#df = df.dropna(axis=1, how='all')

In [7]:
df.shape

(876, 2)

Ahora todas las columnas tienen al menos 1 dato.

# 2) Mails

In [8]:
df_reduced = df.rename(columns={"Documento": "NRO_DOC", 'Email': 'Dato'})

In [9]:
df_reduced.head()

,NRO_DOC,Dato
0,10128300,raulblanco11@hotmail.com
1,10141823,carmenotanez@hotmail.com
2,10460610,juando@gmail.com
3,10497875,caty1951@hotmail.com
4,10580772,fernando.bazan@hotmail.com


# 4) Drop NaNs

Dropeamos aquellas filas que tengan NaN en la columna Dato, ya que no nos aporta nada:

In [10]:
df_melt = df_reduced.dropna(subset=['Dato'])

In [11]:
df_melt.shape

(876, 2)

In [12]:
df_melt.head()

,NRO_DOC,Dato
0,10128300,raulblanco11@hotmail.com
1,10141823,carmenotanez@hotmail.com
2,10460610,juando@gmail.com
3,10497875,caty1951@hotmail.com
4,10580772,fernando.bazan@hotmail.com


# 5) Propietario del Dato

En función de la columna "variable" creamos una nueva columna PROPIETARIO_DATO

In [13]:
df_melt['PROPIETARIO_DATO'] = 'Propio'

In [14]:
df_melt.head()

,NRO_DOC,Dato,PROPIETARIO_DATO
0,10128300,raulblanco11@hotmail.com,Propio
1,10141823,carmenotanez@hotmail.com,Propio
2,10460610,juando@gmail.com,Propio
3,10497875,caty1951@hotmail.com,Propio
4,10580772,fernando.bazan@hotmail.com,Propio


# 6) Columna Proveedor

Agregamos la columna PROVEEDOR_DATO, con el código correspondiente.

PyP = P2

In [15]:
cod_pyp = 'CD'

In [16]:
df_clean = df_melt.assign(PROVEEDOR_DATO=cod_pyp)

In [17]:
df_clean.head()

,NRO_DOC,Dato,PROPIETARIO_DATO,PROVEEDOR_DATO
0,10128300,raulblanco11@hotmail.com,Propio,CD
1,10141823,carmenotanez@hotmail.com,Propio,CD
2,10460610,juando@gmail.com,Propio,CD
3,10497875,caty1951@hotmail.com,Propio,CD
4,10580772,fernando.bazan@hotmail.com,Propio,CD


# 7) Columna Tipo

Agregamos la columna TIPO, con el código correspondiente

In [18]:
tipo_dato = 'EMAIL'

In [19]:
df_clean = df_clean.assign(TIPO=tipo_dato)

In [20]:
df_clean.head ()

,NRO_DOC,Dato,PROPIETARIO_DATO,PROVEEDOR_DATO,TIPO
0,10128300,raulblanco11@hotmail.com,Propio,CD,EMAIL
1,10141823,carmenotanez@hotmail.com,Propio,CD,EMAIL
2,10460610,juando@gmail.com,Propio,CD,EMAIL
3,10497875,caty1951@hotmail.com,Propio,CD,EMAIL
4,10580772,fernando.bazan@hotmail.com,Propio,CD,EMAIL


# 8) Columna Subtipo

Agregamos la columna SUBTIPO, que va a contener información sobre el tipo de mail que se encuentra en el campo Dato.

- Personal: @yahoo, @gmail, @hotmail, @outlook, @live.
- Laboral: el resto.

Verificamos que la string contenga esos casos:

In [21]:
df_clean.loc[(df_clean['Dato'].str.contains('@yahoo', na=False)) | 
             (df_clean['Dato'].str.contains('@gmail', na=False)) |
             (df_clean['Dato'].str.contains('@outlook', na=False)) |
             (df_clean['Dato'].str.contains('@hotmail', na=False)) |
             (df_clean['Dato'].str.contains('@live', na=False)) , 'SUBTIPO'] = 'PERSONAL'

Para los mails laborales, evaluamos los casos contrarios:

In [22]:
df_clean.loc[~(df_clean['Dato'].str.contains('@yahoo', na=False)) & 
             ~(df_clean['Dato'].str.contains('@gmail', na=False)) &
             ~(df_clean['Dato'].str.contains('@outlook', na=False)) &
             ~(df_clean['Dato'].str.contains('@hotmail', na=False)) &
             ~(df_clean['Dato'].str.contains('@live', na=False)) , 'SUBTIPO'] = 'LABORAL'

In [23]:
df_clean.head()

,NRO_DOC,Dato,PROPIETARIO_DATO,PROVEEDOR_DATO,TIPO,SUBTIPO
0,10128300,raulblanco11@hotmail.com,Propio,CD,EMAIL,PERSONAL
1,10141823,carmenotanez@hotmail.com,Propio,CD,EMAIL,PERSONAL
2,10460610,juando@gmail.com,Propio,CD,EMAIL,PERSONAL
3,10497875,caty1951@hotmail.com,Propio,CD,EMAIL,PERSONAL
4,10580772,fernando.bazan@hotmail.com,Propio,CD,EMAIL,PERSONAL


# 9) Columna Score

El campo **Score** se construye en función de los datos obtenidos:

- Nivel 4: Dato que otorga la propia persona.
- Nivel 3: Dato que otorgan los clientes (los bancos).
- Nivel 2: Dato que otorga un proveedor externo (Informes Digitales en este caso), con subtipo "Personal".
- Nivel 1: Dato que otorga un proveedor externo, pero tiene como subtipo "Laboral".

In [24]:
# Nivel 1: Laboral, proveedor externo.

df_clean.loc[(df_clean['SUBTIPO'] == 'LABORAL') , 'SCORE_DATO'] = 1

In [25]:
# Nivel 2: Personal, proveedor externo.

df_clean.loc[(df_clean['SUBTIPO'] == 'PERSONAL') , 'SCORE_DATO'] = 2

In [26]:
df_clean.head()

,NRO_DOC,Dato,PROPIETARIO_DATO,PROVEEDOR_DATO,TIPO,SUBTIPO,SCORE_DATO
0,10128300,raulblanco11@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0
1,10141823,carmenotanez@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0
2,10460610,juando@gmail.com,Propio,CD,EMAIL,PERSONAL,2.0
3,10497875,caty1951@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0
4,10580772,fernando.bazan@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0


# 10) Columna Fecha

Para el campo FECHA_INGRESO_DATO, usamos la fecha en la que se hace la inserción de los datos:

In [27]:
fecha_ingreso = pd.to_datetime('today')

In [28]:
df_clean = df_clean.assign(FECHA_INGRESO_DATO=fecha_ingreso)

In [29]:
df_clean.head()

,NRO_DOC,Dato,PROPIETARIO_DATO,PROVEEDOR_DATO,TIPO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO
0,10128300,raulblanco11@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325
1,10141823,carmenotanez@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325
2,10460610,juando@gmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325
3,10497875,caty1951@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325
4,10580772,fernando.bazan@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325


# 11) Campos adicionales

Creamos las columnas VERIFICADO, FECHA_VERIFICACION, OPERADOR_VERIFICACION, que en principio van en nulls.

In [30]:
df_clean['VERIFICADO'] = 0

In [31]:
df_clean['FECHA_VERIFICACION'] = np.nan

In [32]:
df_clean['OPERADOR_VERIFICACION'] = np.nan

# 12) Concatenación con .csv enviado 

Vamos a concatenar nuestro dataframe con el dataframe enviado para enriquecer. Para esto, vamos a hacer una limpieza inicial de cómo se enviaron esos datos. Finalmente, se uniran ambos dataframes por __DNI__.

## 12.1) Dataset: PyP

Vamos a analizar y limpiar el dataset enviado a PyP para el enriquecimiento:

In [33]:
df_pyp = pd.read_csv('/mnt/c/users/Dleg/Data/MOPC/MOPC/Enriquecimiento de Datos/Data Cleaning/Contacto Directo/Datasets/Datasets enviados/cartera_completa_estado_idcliente - 0103.csv')

In [34]:
df_pyp.head()

,IDMOROSO,DATOADIC,NOMBRE,IDCLIENTE,ESTADO,DNI
0,CUIT20115091280,,SUCESION DE CAISUTTI MIGUEL,BSR_TARDIA,SKIP LEVEL 2,CUIT20115091280
1,CUIT30500181547,,LITOGRAFIA CUYO SA,BSR_TARDIA,PROBLEMA,CUIT30500181547
2,CUIT30501109041,,EMEGE SA,BSR_TARDIA,NO UBICADO,CUIT30501109041
3,CUIT30502016373,,IATILO SA,BSR_TARDIA,NO UBICADO,CUIT30502016373
4,CUIT30502766917,,MAZTER INDUSTR MADERIL SACIFI,BSR_TARDIA,INTERESADO_MOPCPAGOS,CUIT30502766917


Creamos una columna que contenga IDMOROSO + DATOADIC según corresponda

In [35]:
#df_pyp.loc[(df_pyp['IDCLIENTE'].str.startswith('AMEX')) , 'NRO_DOC'] = df_pyp['DATOADIC']

In [36]:
df_pyp = df_pyp [['IDMOROSO','NOMBRE','IDCLIENTE', 'DNI']]

In [37]:
df_pyp = df_pyp.rename(columns={'DNI':'NRO_DOC'})

In [38]:
df_pyp.head()

,IDMOROSO,NOMBRE,IDCLIENTE,NRO_DOC
0,CUIT20115091280,SUCESION DE CAISUTTI MIGUEL,BSR_TARDIA,CUIT20115091280
1,CUIT30500181547,LITOGRAFIA CUYO SA,BSR_TARDIA,CUIT30500181547
2,CUIT30501109041,EMEGE SA,BSR_TARDIA,CUIT30501109041
3,CUIT30502016373,IATILO SA,BSR_TARDIA,CUIT30502016373
4,CUIT30502766917,MAZTER INDUSTR MADERIL SACIFI,BSR_TARDIA,CUIT30502766917


In [39]:
#df_pyp.to_excel('test_nro_doc.xlsx')

## 12.2) DNI Cleaning

Limpiamos el dataframe __df_pyp__, ya que dentro de DATO_A_ENVIAR tenemos 9 tipos de casos diferentes de datos:

- DNI 12345678
- DNI12345678
- DNIE1234678
- LC  1234567
- LE  1234567
- CUIT00123456780
- 00123456780 --> Es un CUIT.
- 00 12345678 0 --> Es un CUIT.
- DNI 00123456780 --> Dice DNI pero es un CUIT de una persona __jurídica__ (TO DO)

Para eso definimos 5 funciones:

- __Tipo_DOC__: Se definen dos funciones para determinar el tipo de DNI. Una inicial, y otra para corregir sobre el dataframe resultante.
- __Standard DNI__: Estandariza los DNI para 7 de los casos. Esta función debe ejecutarse en primer lugar.
- __DNI Spaces__: Elimina los espacios.
- __DNI Zero__: Elimina los ceros iniciales de ciertos CUITs para que coincidan con los DNI.
- __Cuit__: Decisión sobre dejar un cuit o volverlo DNI

In [40]:
def tipo_doc_1 (string_dni):
    
    if 'DNI' in string_dni:
        tipo = 'DNI'
    
    elif 'LC' in string_dni:
        tipo = 'LC'
    
    elif 'LE' in string_dni:
        tipo = 'LE'
    
    elif 'DNIE' in string_dni:
        tipo = 'DNIE'    
    
    elif ((string_dni.startswith(('CUIT30','CUIT33','CUIT34'))) & (len(string_dni)>8)):
        tipo = 'CUIT'
        
    elif ((string_dni.startswith(('30','33','34'))) & (len(string_dni)>8)):
        tipo = 'CUIT'

    elif ((string_dni.startswith(('20','23','24','27'))) & (len(string_dni)>8)):
        tipo = 'DNI'

    elif ((string_dni.startswith(('CUIT20','CUIT23','CUIT24','CUIT27'))) & (len(string_dni)>8)):
        tipo = 'DNI'
        
    else:
        tipo = 'DNI'
    
    return tipo

In [41]:
def tipo_doc_2 (string_dni, tipo_original):
    
    if (len(string_dni) > 8) & (tipo_original == 'DNI'):
        tipo = 'CUIT'
    else:
        tipo = tipo_original
    return tipo

In [42]:
def standard_dni (string_dni):
    """DNI separation by each case.
    """
    
    if 'DNI ' in string_dni:
        dni = string_dni.split('DNI ')[1]
        
    elif 'DNIE' in string_dni:
        dni = string_dni.split('DNIE')[1]
    
    elif 'DNI' in string_dni:
        dni = string_dni.split('DNI')[1]
    
    elif 'LC  ' in string_dni:
        dni = string_dni.split('LC  ')[1]
    
    elif 'LE  ' in string_dni:
        dni = string_dni.split('LE  ')[1]
    
    elif 'CUIT' in string_dni:
        dni = string_dni.split('CUIT')[1]
        
    else:
        dni = string_dni
        
    return dni

In [43]:
def dni_characters (string_dni):
    
    if ' ' in string_dni:
        dni = string_dni.replace(' ','')
        
    elif '-' in string_dni:
        dni = string_dni.replace('-','')

    else:
        dni = string_dni
    
    return dni

In [44]:
def cuit (string_dni):
    
    if ((string_dni.startswith('20')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]
        
    elif ((string_dni.startswith('23')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]
        
    elif ((string_dni.startswith('24')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]

    elif ((string_dni.startswith('27')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]

    else:
        cuit = string_dni
        
    return cuit

In [45]:
def dni_zero (string_dni):
    
    if string_dni.startswith('0'):
        dni = string_dni [1:]

    else:
        dni = string_dni
    
    return dni

En primer lugar, trimmeamos los espacios que sobran:

In [46]:
df_pyp['NRO_DOC'] = [(str(df_pyp['NRO_DOC'][i]).strip()) for i in range (df_pyp['NRO_DOC'].count())]

Se implementan estas funciones:

Para determinar el __TIPO DE DOCUMENTO__ en primer lugar:

In [47]:
df_pyp['TIPO_DOC'] = [tipo_doc_1(df_pyp['NRO_DOC'][i]) for i in range (df_pyp['NRO_DOC'].count())]

Para la limpieza de DNIs:

In [48]:
df_pyp['NRO_DOC'] = [standard_dni(df_pyp['NRO_DOC'][i]) for i in range (df_pyp['NRO_DOC'].count())]

In [49]:
df_pyp['NRO_DOC'] = [dni_characters(df_pyp['NRO_DOC'][i]) for i in range (df_pyp['NRO_DOC'].count())]

In [50]:
df_pyp['NRO_DOC'] = [cuit(df_pyp['NRO_DOC'][i]) for i in range (df_pyp['NRO_DOC'].count())]

In [51]:
df_pyp['NRO_DOC'] = [dni_zero(df_pyp['NRO_DOC'][i]) for i in range (df_pyp['NRO_DOC'].count())]

Para corregir el __TIPO DE DOCUMENTO__ sobre el dataframe resultante

In [52]:
df_pyp['TIPO_DOC'] = [tipo_doc_2(df_pyp['NRO_DOC'][i],df_pyp['TIPO_DOC'][i]) for i in range (df_pyp['NRO_DOC'].count())]

In [53]:
df_pyp.head()

,IDMOROSO,NOMBRE,IDCLIENTE,NRO_DOC,TIPO_DOC
0,CUIT20115091280,SUCESION DE CAISUTTI MIGUEL,BSR_TARDIA,11509128,DNI
1,CUIT30500181547,LITOGRAFIA CUYO SA,BSR_TARDIA,30500181547,CUIT
2,CUIT30501109041,EMEGE SA,BSR_TARDIA,30501109041,CUIT
3,CUIT30502016373,IATILO SA,BSR_TARDIA,30502016373,CUIT
4,CUIT30502766917,MAZTER INDUSTR MADERIL SACIFI,BSR_TARDIA,30502766917,CUIT


# 13) Union de Dataframes

Se unen ambos dataframes en la columna NRO_DOC

Convertimos la columna NRO_DOC de objeto a INT para hacer la unión

In [54]:
df_clean = df_clean.astype({'NRO_DOC': 'str'})

In [55]:
df_union = pd.merge(df_clean,df_pyp, on='NRO_DOC', how='inner')

In [56]:
df_union.shape

(877, 15)

In [57]:
df_union.head()

,NRO_DOC,Dato,PROPIETARIO_DATO,PROVEEDOR_DATO,TIPO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,IDMOROSO,NOMBRE,IDCLIENTE,TIPO_DOC
0,10128300,raulblanco11@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325,0,NaN,NaN,DNI 10128300,BLANCO. RAUL ALFREDO,HSBC_2019,DNI
1,10141823,carmenotanez@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325,0,NaN,NaN,DNI 10141823,OTANEZ LASTRA CARMEN SARA,FRANCES_2012,DNI
2,10460610,juando@gmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325,0,NaN,NaN,DNI 10460610,ANDRADA JUAN DOMINGO,FRANCES_2012,DNI
3,10497875,caty1951@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325,0,NaN,NaN,DNI 10497875,CIURO CATALINA ISABEL,FRANCES_2012,DNI
4,10580772,fernando.bazan@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325,0,NaN,NaN,DNI 10580772,"FERNANDO MARINO,BAZAN",BSR_TARDIA,DNI


## Cuit

Por ahora, quedará en NaN

In [58]:
df_union = df_union.assign(CUIT_CUIL=np.nan)

# 14) Export

Renombramos, dropeamos y reordenamos las columnas

In [59]:
df_union = df_union.rename(columns = {'Dato':'DATO'})

Dropeamos columnas que no nos sirven

In [60]:
df_export = df_union.drop(columns=['IDCLIENTE'], axis=0)

In [61]:
df_export = df_export[['IDMOROSO', 'TIPO_DOC','NRO_DOC','CUIT_CUIL','TIPO','SUBTIPO','DATO','PROPIETARIO_DATO','PROVEEDOR_DATO','FECHA_INGRESO_DATO','VERIFICADO','FECHA_VERIFICACION','OPERADOR_VERIFICACION','SCORE_DATO']]

In [62]:
df_export.head()

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO
0,DNI 10128300,DNI,10128300,NaN,EMAIL,PERSONAL,raulblanco11@hotmail.com,Propio,CD,2021-03-02 16:44:50.000325,0,NaN,NaN,2.0
1,DNI 10141823,DNI,10141823,NaN,EMAIL,PERSONAL,carmenotanez@hotmail.com,Propio,CD,2021-03-02 16:44:50.000325,0,NaN,NaN,2.0
2,DNI 10460610,DNI,10460610,NaN,EMAIL,PERSONAL,juando@gmail.com,Propio,CD,2021-03-02 16:44:50.000325,0,NaN,NaN,2.0
3,DNI 10497875,DNI,10497875,NaN,EMAIL,PERSONAL,caty1951@hotmail.com,Propio,CD,2021-03-02 16:44:50.000325,0,NaN,NaN,2.0
4,DNI 10580772,DNI,10580772,NaN,EMAIL,PERSONAL,fernando.bazan@hotmail.com,Propio,CD,2021-03-02 16:44:50.000325,0,NaN,NaN,2.0


In [63]:
df_export.shape

(877, 14)

Número a str para no tener problemas con la inserción

In [64]:
#df_export['CUIT_CUIL'] = [(str(int(round(df_export['CUIT_CUIL'][i],0)))) for i in range (df_export['CUIT_CUIL'].count())]

In [65]:
df_export.tail()

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO
872,DNIE95470145,DNI,95470145,NaN,EMAIL,PERSONAL,jancv9@hotmail.com,Propio,CD,2021-03-02 16:44:50.000325,0,NaN,NaN,2.0
873,DNIE95570436,DNI,95570436,NaN,EMAIL,PERSONAL,humberto17g@gmail.com,Propio,CD,2021-03-02 16:44:50.000325,0,NaN,NaN,2.0
874,DNIE95632441,DNI,95632441,NaN,EMAIL,PERSONAL,kamilanascimentocosta@gmail.com,Propio,CD,2021-03-02 16:44:50.000325,0,NaN,NaN,2.0
875,DNIE95653471,DNI,95653471,NaN,EMAIL,PERSONAL,bel.badini@outlook.com,Propio,CD,2021-03-02 16:44:50.000325,0,NaN,NaN,2.0
876,DNIE95859283,DNI,95859283,NaN,EMAIL,PERSONAL,nathalyarates@gmail.com,Propio,CD,2021-03-02 16:44:50.000325,0,NaN,NaN,2.0


Dropeamos filas duplicadas

In [66]:
df_export = df_export.drop_duplicates()

In [67]:
df_export.shape

(877, 14)

## Dato a str + strip()

In [68]:
df_export ['DATO'] = df_export['DATO'].str.strip()

# Revisión de duplicados

In [69]:
duplicates_subset = df_export[df_export.duplicated(subset=['IDMOROSO','DATO','PROVEEDOR_DATO'],keep=False)]

In [70]:
duplicates_subset

,IDMOROSO,TIPO_DOC,NRO_DOC,CUIT_CUIL,TIPO,SUBTIPO,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,SCORE_DATO


In [71]:
duplicates_subset.shape

(0, 14)

No tenemos duplicados en IDMOROSO, DATO, PROVEEDOR

In [72]:
df_export.shape

(877, 14)

## 14.1) Export de dataframe + IDCLIENTE para análisis

In [73]:
#df_union.to_excel('D:/MOPC/Enriquecimiento de Datos/Data Cleaning/PyP/Exports/Mails para analisis/PyP_mails_1401.xlsx')
#df_union.to_csv('D:/MOPC/Enriquecimiento de Datos/Data Cleaning/PyP/Exports/Mails para analisis/PyP_mails_1401.csv')

## 14.2) Insert a tabla DAM_CONTACTABILIDAD:

In [74]:
#df_export.to_csv('/mnt/c/users/Dleg/Data/MOPC/MOPC/Enriquecimiento de Datos/Data Cleaning/Contacto Directo/Exports/DAM_CONTACTABILIDAD/CD_DAM_CONTACTABILIDAD_mails_0103.csv',index=False)

In [75]:
df_union.shape

(877, 16)

In [76]:
df_union ['DATO'] = df_union['DATO'].str.strip()

In [77]:
df_union = df_union.drop_duplicates()

In [78]:
df_union.shape

(877, 16)

In [79]:
df_union.head()

,NRO_DOC,DATO,PROPIETARIO_DATO,PROVEEDOR_DATO,TIPO,SUBTIPO,SCORE_DATO,FECHA_INGRESO_DATO,VERIFICADO,FECHA_VERIFICACION,OPERADOR_VERIFICACION,IDMOROSO,NOMBRE,IDCLIENTE,TIPO_DOC,CUIT_CUIL
0,10128300,raulblanco11@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325,0,NaN,NaN,DNI 10128300,BLANCO. RAUL ALFREDO,HSBC_2019,DNI,NaN
1,10141823,carmenotanez@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325,0,NaN,NaN,DNI 10141823,OTANEZ LASTRA CARMEN SARA,FRANCES_2012,DNI,NaN
2,10460610,juando@gmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325,0,NaN,NaN,DNI 10460610,ANDRADA JUAN DOMINGO,FRANCES_2012,DNI,NaN
3,10497875,caty1951@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325,0,NaN,NaN,DNI 10497875,CIURO CATALINA ISABEL,FRANCES_2012,DNI,NaN
4,10580772,fernando.bazan@hotmail.com,Propio,CD,EMAIL,PERSONAL,2.0,2021-03-02 16:44:50.000325,0,NaN,NaN,DNI 10580772,"FERNANDO MARINO,BAZAN",BSR_TARDIA,DNI,NaN


In [80]:
df_union.IDCLIENTE.value_counts()

HSBC_2019               332
FRANCES_2012            285
BSR_TARDIA              197
BBVA_LEGALES_PREJU       21
HSBC_LEGALES14           18
FRAVEGA_LEGALES548       16
COTO23_TARD               3
COTO05_TARD               1
COTO10_TARD               1
HSBC_LEGALES_TEMP         1
AMEX_PARM                 1
AMEX_QINM                 1
Name: IDCLIENTE, dtype: int64